In [1]:
"""
Lists the available Amazon Bedrock models in an &AWS-Region;.
"""
import logging
import json
import boto3


from botocore.exceptions import ClientError


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def list_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models.

    :return: The list of available bedrock foundation models.
    """

    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        logger.info("Got %s foundation models.", len(models))
        return models

    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise


def main():
    """Entry point for the example. Change aws_region to the &AWS-Region;
    that you want to use."""
   
    aws_region = "us-east-1"

    bedrock_client = boto3.client(service_name="bedrock", region_name=aws_region)
    
    fm_models = list_foundation_models(bedrock_client)
    for model in fm_models:
        print(f"Model: {model["modelName"]}")
        print(json.dumps(model, indent=2))
        print("---------------------------\n")
    
    logger.info("Done.")

# if __name__ == "__main__":
main()


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:__main__:Got 124 foundation models.
INFO:__main__:Done.


Model: NVIDIA Nemotron Nano 12B v2 VL BF16
{
  "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/nvidia.nemotron-nano-12b-v2",
  "modelId": "nvidia.nemotron-nano-12b-v2",
  "modelName": "NVIDIA Nemotron Nano 12B v2 VL BF16",
  "providerName": "NVIDIA",
  "inputModalities": [
    "TEXT",
    "IMAGE"
  ],
  "outputModalities": [
    "TEXT"
  ],
  "responseStreamingSupported": true,
  "customizationsSupported": [],
  "inferenceTypesSupported": [
    "ON_DEMAND"
  ],
  "modelLifecycle": {
    "status": "ACTIVE"
  }
}
---------------------------

Model: Claude Sonnet 4
{
  "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-sonnet-4-20250514-v1:0",
  "modelId": "anthropic.claude-sonnet-4-20250514-v1:0",
  "modelName": "Claude Sonnet 4",
  "providerName": "Anthropic",
  "inputModalities": [
    "TEXT",
    "IMAGE"
  ],
  "outputModalities": [
    "TEXT"
  ],
  "responseStreamingSupported": true,
  "customizationsSupported": [],
  "inferenceTypesSupported": [
    

In [ ]:
# Use the native inference API to send a text message to Anthropic Claude.

import boto3
import json

from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Claude 3 Haiku.
model_id = "anthropic.claude-3-haiku-20240307-v1:0"



# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload using the model's native structure.
native_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "temperature": 0.5,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = client.invoke_model(modelId=model_id, body=request)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["content"][0]["text"]
print(response_text)




The purpose of a 'hello world' program is to serve as a simple and straightforward introduction to a programming language or development environment, demonstrating the basic syntax and execution of a basic program.


In [13]:
def interact_with_bedrock_model(model_id,prompt,temperature):

    'generates response interacting with bedrock models'

    # Format the request payload using the model's native structure.
    native_request = {
        "inputText": prompt,
        "textGenerationConfig": {
            "maxTokenCount": 512,
            "temperature": 0.5,
        },
    }


    request = json.dumps(native_request)


    response = client.invoke_model(modelId=model_id, body=request)

    response_text = model_response["content"][0]["text"]

    return response_text

    



In [20]:
import boto3

bedrock = boto3.client("bedrock", region_name="us-east-1")

response = bedrock.list_foundation_models()
models = response["modelSummaries"]


In [21]:
active_models = [
    m for m in models
    if m.get("modelLifecycle", {}).get("status") == "ACTIVE"
]


In [22]:
selected_models = {}

for model in active_models:
    provider = model["providerName"]

    if provider not in selected_models:
        selected_models[provider] = model["modelId"]


In [23]:
final_selection = {
    "Amazon Titan": selected_models.get("Amazon"),
    "Anthropic Claude": selected_models.get("Anthropic"),
    "Meta Llama": selected_models.get("Meta"),
    "Mistral": selected_models.get("Mistral AI")
}

for category, model_id in final_selection.items():
    print(f"{category}: {model_id}")


Amazon Titan: amazon.nova-2-multimodal-embeddings-v1:0
Anthropic Claude: anthropic.claude-sonnet-4-20250514-v1:0
Meta Llama: meta.llama3-8b-instruct-v1:0
Mistral: mistral.voxtral-mini-3b-2507


### DEMO 1 — Same Prompt, Different Models

In [14]:
prompt = ''' 
You are an expert technical writer.

Task:
Summarize the following content for a cloud engineering audience.

Constraints:
- Maximum 5 bullet points
- Keep it concise
- Focus on key benefits

Content:
"""
Amazon Bedrock is a fully managed service that provides access to
foundation models via a single API. It enables developers to build
Generative AI applications without managing infrastructure...
"""


'''



In [15]:
model_id = 'meta.llama3-8b-instruct-v1:0'

temperature = 0.2


print(interact_with_bedrock_model(model_id,prompt,temperature))

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: required key [prompt] not found#: extraneous key [inputText] is not permitted#: extraneous key [textGenerationConfig] is not permitted, please reformat your input and try again.

In [26]:
import boto3
import json

# Ensure you have the correct IAM permissions (bedrock:InvokeModel)
# and model access is granted in the AWS console.

def invoke_bedrock_model(prompt, model_id='anthropic.claude-v2'):
    """
    Invokes a specified Amazon Bedrock model with a given prompt.

    :param prompt: The prompt to send to the model.
    :param model_id: The ID of the model to use.
    :return: The generated response text.
    """
    try:
        # Initialize the bedrock-runtime client
        client = boto3.client('bedrock-runtime', region_name='us-east-1')

        # The request body format varies by model.
        # This example is for Anthropic Claude models.
        body = json.dumps({
            "prompt": f"Human: {prompt}\n\nAssistant:",
            "temperature": 0.5
        })

        response = client.invoke_model(
            body=body,
            modelId=model_id,
            accept='application/json',
            contentType='application/json'
        )

        response_body = json.loads(response.get('body').read())
        return response_body.get('completion')

    except Exception as e:
        print(f"Error invoking model: {e}")
        return None

# Example usage
prompt = "What is the capital of France?"
response_text = invoke_bedrock_model(prompt,'amazon.titan-tg1-large')
if response_text:
    print(f"Model Response: {response_text}")


Error invoking model: An error occurred (ResourceNotFoundException) when calling the InvokeModel operation: This model version has reached the end of its life. Please refer to the AWS documentation for more details.


In [9]:
response_text

In [16]:
import boto3
import json

bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")


def invoke_bedrock(model_id: str, prompt: str, temperature: float = 0.5):
    """
    Unified Bedrock invocation for Titan, Claude, Llama, and Mistral
    """

    # -------- Payload Builder -------- #
    if "amazon.titan" in model_id:
        body = {
            "inputText": prompt,
            "textGenerationConfig": {
                "temperature": temperature,
                "maxTokenCount": 300,
                "topP": 0.9
            }
        }

    elif "anthropic.claude" in model_id:
        body = {
            "prompt": f"\n\nHuman: {prompt}\n\nAssistant:",
            "temperature": temperature,
            "max_tokens_to_sample": 300
        }

    elif "meta.llama" in model_id:
        body = {
            "prompt": prompt,
            "temperature": temperature,
            "max_gen_len": 300
        }

    elif "mistral" in model_id:
        body = {
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": 300
        }

    else:
        raise ValueError(f"Unsupported model: {model_id}")

    # -------- Invoke Model -------- #
    response = bedrock.invoke_model(
        modelId=model_id,
        body=json.dumps(body),
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response["body"].read())

    # -------- Normalize Output -------- #
    if "amazon.titan" in model_id:
        return response_body["results"][0]["outputText"]

    elif "anthropic.claude" in model_id:
        return response_body["completion"]

    elif "meta.llama" in model_id:
        return response_body["generation"]

    elif "mistral" in model_id:
        return response_body["outputs"][0]["text"]


In [ ]:
prompt = "Summarize Amazon Bedrock in 3 bullet points."

models = {'titan': 'amazon.titan-tg1-large', 
'claude': 'anthropic.claude-sonnet-4-20250514-v1:0', 
'llama': 'meta.llama3-8b-instruct-v1:0', 
'mistral': 'mistral.voxtral-mini-3b-2507'}

for model,model_id in models.items():
    print(f"\n--- {model} ---")
    print(invoke_bedrock(model_id, prompt, temperature=0.3))



--- anthropic.claude-3.5-sonnet-20240620-v1:0 ---


AttributeError: 'Bedrock' object has no attribute 'invoke_model'